# VAE–LSTM (ACN Data)

In [ ]:
#set seeds for deterministic behavior (as much as possible)
import os
import random
import numpy as np
import tensorflow as tf

SEED = 42

def set_seed(seed: int = SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(SEED)

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *


## 2. Configuration

In [ ]:
# Paths
DATA_PATH = "dataset.csv"   # <-- change to your local path (recommended: data/dataset.csv)

# Train/test split
train_size = 0.8
val_split = 0.2

# Sequence settings
sequence_length = 24*7     # one week of hourly data       
missing_len = 24           # to deconstruct the sequence

# Masking NaNs
mask_value = -999.      

# Model/training 
nn_size = 32
BATCH_SIZE = 168         # one week of hourly data    
EPOCHS = 100
lr_value = 1e-3
latent_dim = 2           # to be able to plot in 2D easily

# Plotting only
seq_id_1 = 7000               # Select sequence number 7000
seq_id_2 = 2500               # Select sequence number 2500
id_seq = 11595                # sequence ID to test on


## 3. Data loading

### Read Data

In [ ]:
df = pd.read_csv(DATA_PATH)
df['Date_Time'] = pd.to_datetime(df['Date_Time'])
df.drop_duplicates('Date_Time', inplace=True)
df.set_index('Date_Time', inplace=True)

print(df.shape)

In [ ]:
### INSERT MISSING DATES ### (between Dec 15 2018 to Jan 1 2019)

df = df.reindex(pd.date_range(df.head(1).index[0], df.tail(1).index[0], freq='H'))

print(df.shape)

### Plot Traffic Sample

In [ ]:
df.kWh_target.tail(1000).plot(figsize=(18,5))     #last 1000 samples without missing values
plt.ylabel('kWh per hour')

### Plot Missing Values Over Time

In [ ]:
plt.figure(figsize=(18,5))
sns.heatmap(df[['kWh_target']].isna().T, cbar=False, cmap='plasma', 
            xticklabels=False, yticklabels=['traffic NaNs'])
plt.xticks(range(0,len(df), 24*180), list(df.index.year[::24*180]))  # a tick every 6 months (24 hours a day, 180 days)
np.set_printoptions(False)

### Fill Missing Values

In [ ]:
df = df[df.index.year.isin([2018,2019])].copy()

print(df['Solar'].isnull().values.any())            # still with missing values 

In [ ]:

df = pd.concat([df.select_dtypes(include=['object']).fillna(method='backfill'),
                df.select_dtypes(include=['float']).interpolate(method='time')], axis=1)

print(df.shape)


### Plot Traffic Distribution In Each Month

In [ ]:
plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.month, y=df.kWh_target, palette='plasma')

plt.ylabel('kWh per hour'); plt.xlabel('month')

### Plot Traffic Distribution In Each Weekday

In [ ]:
plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.weekday, y=df.kWh_target, palette='plasma')

plt.ylabel('kWh per hour'); plt.xlabel('weekday')

### Plot Traffic Distribution In Each Hour

In [ ]:
plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.hour, y=df.kWh_target, palette='plasma')

plt.ylabel('kWh per hour'); plt.xlabel('hour')

In [ ]:
X = df.drop(columns=['kWh_target'])
print(X)

## 4. Data Preparation

### Add Temporal Features

In [ ]:
map_col = dict()
map_col = {'Rain': 0, 'Temp': 1, 'Solar': 2, 'kWh_input': 3}
i=3                                                              # 4 features infront 
X['month'] = df.index.month;  i += 1;  map_col['month'] = i
X['weekday'] = df.index.weekday;  i += 1;  map_col['weekday'] = i
X['hour'] = df.index.hour;  i += 1;  map_col['hour'] = i
X.shape
print(X)

### Function For 3D Sequence Generation

In [ ]:
def gen_seq(id_df, seq_length, seq_cols):

    data_matrix =  id_df[seq_cols]
    num_elements = data_matrix.shape[0]

    for start, stop in zip(range(0, num_elements-seq_length, 1), range(seq_length, num_elements, 1)):
        
        yield data_matrix[stop-sequence_length:stop].values.reshape((-1,len(seq_cols)))

### Generate 3D Sequences

In [ ]:
sequence_input = []
sequence_target = []

for seq in gen_seq(X, sequence_length, X.columns):
    sequence_input.append(seq)
    
for seq in gen_seq(df, sequence_length, ['kWh_target']):
    sequence_target.append(seq)
    
sequence_input = np.asarray(sequence_input)
sequence_target = np.asarray(sequence_target)

print(sequence_input.shape)
print(sequence_target.shape)

print(sequence_input[1])
print(sequence_target[1])

### Function To Insert Missing Intervals (Randomly)

In [ ]:
def drop_fill_pieces(sequence_input, sequence_target, sequence_length, missing_len, map_col, missing_val=np.nan, size=0.6):
    
    sequence_input = np.copy(sequence_input)
    sequence_target = np.copy(sequence_target)
    
    _id_seq = np.random.choice(range(len(sequence_target)), int(len(sequence_target)*size), replace=False)
    _id_time = np.random.randint(0,sequence_length-missing_len, int(len(sequence_target)*size))
    
    for i,t in zip(_id_seq, _id_time):
        sequence_input[i, t:t+missing_len, 
                       [map_col['Rain'], 
                        map_col['Temp'],
                        map_col['Solar']]] = -1
        sequence_target[i, t:t+missing_len, :] = missing_val
        
    sequence_input[:,:, 
                   [map_col['Rain'], 
                    map_col['Temp'],
                    map_col['Solar']]] += 1
    
    return sequence_input, sequence_target

### Insert Missing Intervals (Randomly)

In [ ]:

sequence_input, sequence_target_drop = drop_fill_pieces(sequence_input, sequence_target, sequence_length,
                                                        missing_len=missing_len, map_col=map_col, size=0.6)

sequence_input.shape, sequence_target_drop.shape

In [ ]:
# Index of missing values only in targets

aa = np.argwhere(np.isnan(sequence_target_drop))
aa[70:150] 
                                      

### Train Test Split

In [ ]:
sequence_input_train = sequence_input[:int(len(sequence_input)*train_size)]
sequence_input_test = sequence_input[int(len(sequence_input)*train_size):]
print(sequence_input_train.shape, sequence_input_test.shape)

sequence_target_train = sequence_target[:int(len(sequence_target)*train_size)]
sequence_target_test = sequence_target[int(len(sequence_target)*train_size):]
print(sequence_target_train.shape, sequence_target_test.shape)

sequence_target_drop_train = sequence_target_drop[:int(len(sequence_target_drop)*train_size)]
sequence_target_drop_test = sequence_target_drop[int(len(sequence_target_drop)*train_size):]
print(sequence_target_drop_train.shape, sequence_target_drop_test.shape)

### Class For Sequences Scaling

In [ ]:
class Scaler1D:
    
    def fit(self, X):
        self.mean = np.nanmean(np.asarray(X).ravel())
        self.std = np.nanstd(np.asarray(X).ravel())
        return self
        
    def transform(self, X):
        return (X - self.mean)/self.std
    
    def inverse_transform(self, X):
        return (X*self.std) + self.mean

### Scale Sequences And Mask Nans

In [ ]:
scaler_target = Scaler1D().fit(sequence_target_train)

sequence_target_train = scaler_target.transform(sequence_target_train)
sequence_target_test = scaler_target.transform(sequence_target_test)

sequence_target_drop_train = scaler_target.transform(sequence_target_drop_train)
sequence_target_drop_test = scaler_target.transform(sequence_target_drop_test)

sequence_target_drop_train[np.isnan(sequence_target_drop_train)] = mask_value
sequence_target_drop_test[np.isnan(sequence_target_drop_test)] = mask_value

In [ ]:
# Checking

map_col.items()

## 5. Variational Autoencoder (VAE)

### Functions For VAE

In [ ]:
# VAE Reparameterization (Samples a latent vector z from N(z_mean, sigma))
def sampling(args):
    
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

# VAE Loss Function (Total loss = Reconstruction loss + KL divergence)
def vae_loss(inp, original, out, z_log_sigma, z_mean):
    
    reconstruction = K.mean(K.square(original - out)) * sequence_length
    kl = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))

    return reconstruction + kl

# VAE Model
def get_model():
    
    set_seed(SEED)
    
    ### encoder ###
    inp = Input(shape=(sequence_length, 1))
    inp_original = Input(shape=(sequence_length, 1))
    
    cat_inp = []
    cat_emb = []
    for cat,i in map_col.items():
        inp_c = Input(shape=(sequence_length,))
        #inp_c = K.cast(inp_c,"int32")
        if cat in ['Rain', 'Temp', 'Solar', 'kWh_input']:
            emb = Embedding(int(X[cat].max()+2), 6)(inp_c)
        else:
            emb = Embedding(int(X[cat].max()+1), 6)(inp_c)
        cat_inp.append(inp_c)
        cat_emb.append(emb)
    
    concat = Concatenate()(cat_emb + [inp])
    #enc = LSTM(nn_size, return_sequences=True)(concat)
    enc = LSTM(nn_size)(concat)
    
    z = Dense(nn_size, activation="relu")(enc)
        
    z_mean = Dense(latent_dim)(z)
    z_log_sigma = Dense(latent_dim)(z)
            
    encoder = Model(cat_inp + [inp], [z_mean, z_log_sigma])
    
    ### decoder ###
    inp_z = Input(shape=(latent_dim,))

    dec = RepeatVector(sequence_length)(inp_z)
    dec = Concatenate()([dec] + cat_emb)
    #dec = LSTM(nn_size, return_sequences=True)(dec)
    dec = LSTM(4, return_sequences=True)(dec)
    
    out = TimeDistributed(Dense(1))(dec)
    
    decoder = Model([inp_z] + cat_inp, out)   
    
    ### encoder + decoder ###
    z_mean, z_log_sigma = encoder(cat_inp + [inp])
    z = Lambda(sampling)([z_mean, z_log_sigma])
    pred = decoder([z] + cat_inp)
    
    vae = Model(cat_inp + [inp, inp_original], pred)
    vae.add_loss(vae_loss(inp, inp_original, pred, z_log_sigma, z_mean))
    vae.compile(loss=None, optimizer=Adam(lr_value))
    
    return vae, encoder, decoder

### Model Training

In [ ]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='loss', mode='auto', restore_best_weights=True)

vae, enc, dec = get_model()
vae.fit([sequence_input_train[:,:,i] for cat,i in map_col.items()] + [sequence_target_drop_train, sequence_target_train], 
         batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split = val_split, shuffle=False, callbacks=[es])  


### Compute Reconstruction

In [ ]:
vae = Model(vae.input[:-1], vae.output)

reconstruc_train = scaler_target.inverse_transform(
    vae.predict([sequence_input_train[:,:,i] for cat,i in map_col.items()] + [sequence_target_drop_train]))
reconstruc_test = scaler_target.inverse_transform(
    vae.predict([sequence_input_test[:,:,i] for cat,i in map_col.items()] + [sequence_target_drop_test]))

reconstruc_train.shape, reconstruc_test.shape

### Plot Real Vs Reconstruction

In [ ]:
seq = np.copy(sequence_target_drop_train[seq_id_1])
seq[seq == mask_value] = np.nan
seq = scaler_target.inverse_transform(seq)

plt.figure(figsize=(9,5))
plt.plot(reconstruc_train[seq_id_1], label='reconstructed', c='red')
plt.plot(seq, c='blue', label='original', alpha=0.6)
plt.legend()

### Plot Real Vs Reconstruction

In [ ]:
seq = np.copy(sequence_target_drop_test[seq_id_2])
seq[seq == mask_value] = np.nan
seq = scaler_target.inverse_transform(seq)

plt.figure(figsize=(9,5))
plt.plot(reconstruc_test[seq_id_2], label='reconstructed', c='red')
plt.plot(seq, c='blue', label='original', alpha=0.6)
plt.legend()

### Compute Performances (On Training)

In [ ]:
mask = (sequence_target_drop_train == mask_value)

print('reconstruction error on entire sequences:',
    mse(np.squeeze(reconstruc_train, -1), np.squeeze(sequence_target_train, -1), squared=False))
print('reconstruction error on missing sequences:',
    mse(reconstruc_train[mask].reshape(-1,missing_len), sequence_target_train[mask].reshape(-1,missing_len), squared=False))

### Compute Performances (On Testing)

In [ ]:
mask = (sequence_target_drop_test == mask_value)

print('reconstruction error on entire sequences:',
    mse(np.squeeze(reconstruc_test, -1), np.squeeze(sequence_target_test, -1), squared=False))
print('reconstruction error on missing sequences:',
    mse(reconstruc_test[mask].reshape(-1,missing_len), sequence_target_test[mask].reshape(-1,missing_len), squared=False))

### Get Latent Representation On Train Data

In [ ]:
enc_pred, _ = enc.predict([sequence_input_train[:,:,i] for cat,i in map_col.items()] + [sequence_target_drop_train])
enc_pred.shape

### Plot Latent Representation

In [ ]:
for cat,i in map_col.items():
    plt.scatter(enc_pred[:,0], enc_pred[:,1], c=sequence_input_train[:,sequence_length//2,i], cmap='plasma')
    plt.title(cat); plt.show()

### Generate Random Permutation

In [ ]:
_X = np.random.normal(enc_pred[id_seq,0], 3, 10)
_Y = np.random.normal(enc_pred[id_seq,1], 3, 10)
_cat_input = [sequence_input_train[[id_seq],:,i] for cat,i in map_col.items()]

In [ ]:
# Checking

print(len(_cat_input))

aa = 0
for x in _X:
    for y in _Y:
        print(np.asarray([[x,y]]))
        aa = aa+1
aa        

### Plot Random Permutation

In [ ]:
bb = 0
plt.figure(figsize=(9,5))
        
for x in _X:
    for y in _Y:
        dec_pred = dec.predict([np.asarray([[x,y]])] + _cat_input)
        plt.plot(scaler_target.inverse_transform(dec_pred[0]), c='orange', alpha=0.6)
        bb = bb +1
        
plt.plot(scaler_target.inverse_transform(sequence_target_train[id_seq]), c='blue')
plt.legend(['Generated Data'])
plt.title('Generated Data vs True Data')
plt.ylabel('kWDelivered values')
plt.xlabel('Data points in hours')

In [ ]:
# Checking 

print(dec_pred.size)
print(bb)